   # Within-fire variability 
   
   Workflow to extract pixels from imagery and boostrap samples to get median and mean for each image.

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import dask
%matplotlib inline
%tb


In [359]:
# required functions
%run processing_functions.py
%run ../pixel_to_df.py

## Test workflow on single fire

What is the distribution of pixel counts in  defol/non-defoliated between and within fires?


Goals: 
1) create reference distribution from one of defoliation / non-defoiated halves
2) from the opposite half, subsample to match sample size of reference. 
3) boostrap resample to get CI and distrbution of values. 
4) compare distribution

Steps:
1) get coordinates and return them as a dataframe with x and y
2) add column to df which says whether it is defoliated or non-defoliated (binary)
3) extract pixel values? 
4) randomly select a subsample of pixels from df, get values...etc.?

* turn into 4d array with time, band, x, y - band would be nbr and defoliated (1,0)


NOTES:
- values to be standardized when in DF format after pixel extraction

In [134]:
# read in data and get coordinates
ds = rx.open_rasterio("/Users/jgoldman/Desktop/chp3/clipped/nbr/non-defoliated/COC16_1997_1038.nc", masked=True)


In [360]:
# random sample for a given year.
# for each sample take the coordinates and get values across df.
def random_sample(da,
                  n, 
                  dim_slice,
                  keep_cols=True):
    """
    take a data array, turn to df and get random sample
    da = xarray data array with n dimensions
    n = number of samples
    dim_slice = slice along certain time dimension
    keep_cols = 

    """
    # slice along dimension
    da2= da.isel(time=dim_slice)
    #to dataframe
    df = da2.to_dataframe()

    # sample location
    samples = []
    no_of_points = n
    # random sample of n points
    sample_loc = df.dropna().sample(n=int(round(no_of_points)))
    samples.append(sample_loc)
    #join back into single datafame
    all_samples = pd.concat([samples[i] for i in range(0,len(samples))])
    
    # x and y to lat and lon column
    all_samples.reset_index(inplace = True)
    all_samples = all_samples.rename(columns={'y': 'Lat', 'x': 'Long'}) 
    
    if keep_cols==False:
        #remove crs, nbr and time cols
        all_samples = all_samples.drop(columns =['crs', 'time', 'nbr'])
     # for each lat/long select pixel
    pix_samples = []
    for index, row in all_samples.iterrows():
        x = row['Long']
        y = row['Lat']
        df2 = da.to_dataframe()
        df2 = df2.dropna()
        df2 = std_nbr(df2)
        df2 = df2.loc[(df2['y'] == y) & (df2['x'] == x)]
        # add add row that goes preNBR - nbr 10
        ints = np.arange(0,12,1)
        df2['nbr_time'] = ints
        df2['nbr_time'] = df2['nbr_time'].replace({'0': 'preNBR', '1': 'nbrYOF', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8': '7', '9':'8', '10': '9', '11': '10'})
        pix_samples.append(df2)

    #join back into single datafame
    all_samples = pd.concat([pix_samples[i] for i in range(0,len(pix_samples))])
  
    #remove crs
    all_samples = all_samples.drop(columns =['crs'])
    
    ids = np.arange(1,1001,1)
    # add pixel id
    all_samples['id'] = np.repeat(ids, 12, axis = 0)
    
    return(all_samples)

ERRORS IN /1000 (std_nbr) at row nbr_time 1

In [ ]:
df = random_sample(ds, 1000, 1, False)
df



In [334]:
df.iloc[13:24, 0:10]
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.iloc[13:24, 0:10]


,year,y,x,nbr,nbr_time,id
49596,1997,49.888,-80.548,-0.103,1,2
79778,1998,49.888,-80.548,-0.000,2,2
109960,1999,49.888,-80.548,0.000,3,2
140142,2000,49.888,-80.548,0.000,4,2
170324,2001,49.888,-80.548,0.000,5,2
200506,2002,49.888,-80.548,0.000,6,2
230688,2003,49.888,-80.548,0.000,7,2
260870,2004,49.888,-80.548,0.001,8,2
291052,2005,49.888,-80.548,0.001,9,2
321234,2006,49.888,-80.548,0.000,10,2


In [303]:
-34.753365 / 1000

-0.034753365

In [314]:
def magnitude_calc(x):
    pre= x[x['nbr_time'] == 0]
    pre = pre['nbr'].item()
    post= x[x['nbr_time'] == 1]
    post = post['nbr'].item()
    p_diff = pre - post
    rec = x[x['nbr_time'] == 11]
    rec = rec['nbr'].item()
    rec_diff = rec - post
    mag = (rec - post)* 100
    
    return pd.DataFrame({'recovery_magnitude': [mag]})



In [315]:
def recovery_magnitude(df):
    grp = df.groupby('id')
    magnitudes = []
    # for length in 1-1001
    ids = np.arange(1,1001,1)
    for i in ids:
        # get group
        x = grp.get_group(i)
        # calc mac
        res = magnitude_calc(x)
        # add id column
        res['id'] = i
        # append to list
        magnitudes.append(res)
        
        
    #join back into single datafame
    all_mags = pd.concat([magnitudes[i] for i in range(0,len(magnitudes))])
    
    return(all_mags)


In [316]:
recovery_magnitude(df)

,recovery_magnitude,id
0,15.619434,1
0,3585.347551,2
0,23600.507420,3
0,-7.109806,4
0,-2.095884,5
...,...,...
0,3034.888580,996
0,-7.486275,997
0,19.601502,998
0,21.116942,999


In [318]:
grp = df.groupby('id')
grp.get_group(2)

,year,y,x,nbr,nbr_time,id
7569,1996,49.911879,-80.570392,0.630455,0,2
37751,1997,49.911879,-80.570392,-35.542747,1,2
67933,1998,49.911879,-80.570392,-0.000916,2,2
98115,1999,49.911879,-80.570392,0.033711,3,2
128297,2000,49.911879,-80.570392,0.068338,4,2
158479,2001,49.911879,-80.570392,0.102966,5,2
188661,2002,49.911879,-80.570392,0.137593,6,2
218843,2003,49.911879,-80.570392,0.172220,7,2
249025,2004,49.911879,-80.570392,0.206847,8,2
279207,2005,49.911879,-80.570392,0.241474,9,2
